# Exploratory Data Analysis (EDA)

- Comparing clean and raw dataset to understand the discrepancy in autistic sample
---
- The cleaned dataset is a carefully matched case-control study
- Only 1,450 autistic participants were included in the final analysis (likely the ones with complete data and good matches)
- The remaining autistic participants were excluded due to:
- Incomplete data
- Inability to find appropriate control matches
- Quality control criteria



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [ ]:
# Path to the large CSV file
csv_path = '/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv'

# Read a small sample to infer dtypes and preview
sample = pd.read_csv(csv_path, nrows=5000)
display(sample.info())
display(sample.head())

In [ ]:
# Try to load the full dataset
try:
    df = pd.read_csv(csv_path)
    print('Shape:', df.shape)
    print('Columns:', df.columns.tolist())
    print('Memory usage (MB):', df.memory_usage(deep=True).sum() / 1e6)
    display(df.head())
    display(df.tail())
    print('Missing values per column:')
    print(df.isnull().sum())
    display(df.describe(include="all"))
except MemoryError:
    print('MemoryError: Consider loading with chunksize or specifying dtypes.')
    df = None

# Counting autism diagnoses

- this is pre-cleaning

In [ ]:
# List of relevant columns
diagnosis_cols = [f'diagnosis_{i}' for i in range(9)]
autism_diag_cols = [f'autism_diagnosis_{i}' for i in range(3)]

# Ensure columns exist in the DataFrame
diagnosis_cols = [col for col in diagnosis_cols if col in df.columns]
autism_diag_cols = [col for col in autism_diag_cols if col in df.columns]

# Condition 1: Any '2' in diagnosis columns
diagnosis_autistic = df[diagnosis_cols].apply(lambda row: (row == 2).any() or (row == '2').any(), axis=1)

# Condition 2: Any '1', '2', or '3' in autism_diagnosis columns
autism_diag_autistic = df[autism_diag_cols].apply(lambda row: row.isin([1, 2, 3]).any() or row.isin(['1', '2', '3']).any(), axis=1)

# Combine conditions
autistic_participants = diagnosis_autistic | autism_diag_autistic

# Count
num_autistic = autistic_participants.sum()
print(f'Number of autistic participants: {num_autistic}')

# Exploring clean dataset 

In [ ]:
# load the cleaned data
clean_csv_path = '/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_clean.csv'
df_clean = pd.read_csv(clean_csv_path)

In [ ]:
# reapeat autistic participants count logic 
diagnosis_cols = [f'diagnosis_{i}' for i in range(9) if f'diagnosis_{i}' in df_clean.columns]
autism_diag_cols = [f'autism_diagnosis_{i}' for i in range(3) if f'autism_diagnosis_{i}' in df_clean.columns]

diagnosis_autistic = df_clean[diagnosis_cols].apply(lambda row: (row == 2).any() or (row == '2').any(), axis=1)
autism_diag_autistic = df_clean[autism_diag_cols].apply(lambda row: row.isin([1, 2, 3]).any() or row.isin(['1', '2', '3']).any(), axis=1)
autistic_participants = diagnosis_autistic | autism_diag_autistic
num_autistic = autistic_participants.sum()
print(f'Number of autistic participants in cleaned data: {num_autistic}')

# comparing discrepancy between datasets

In [ ]:
#compare sizes and columns of raw and clean data

print("Raw dataset shape:", df.shape)
print("Cleaned dataset shape:", df_clean.shape)
print("Raw columns:", df.columns.tolist())
print("Cleaned columns:", df_clean.columns.tolist())

#compare value counts 
for col in diagnosis_cols + autism_diag_cols:
    if col in df.columns and col in df_clean.columns:
        print(f"{col} value counts (raw):")
        print(df[col].value_counts(dropna=False))
        print(f"{col} value counts (clean):")
        print(df_clean[col].value_counts(dropna=False))
        print("-" * 40)

In [ ]:
# checking for deduplication 
# Check unique participants in both datasets
print("Unique userids in raw:", df['userid'].nunique())
print("Unique userids in clean:", df_clean['userid'].nunique())
print("Total rows in raw:", len(df))
print("Total rows in clean:", len(df_clean))

#check filtering column 
print("autismvscontrols_1450matched value counts:")
print(df_clean['autismvscontrols_1450matched'].value_counts(dropna=False))

# Check if participants with autism diagnoses had more missing data
autism_participants_raw = df[df['diagnosis_0'] == 2.0]
print("Missing data in autism participants (raw):")
print(autism_participants_raw.isnull().sum())

# check the matching column (new column in clean data)
print("autismvscontrols_1450matched distribution:")
print(df_clean['autismvscontrols_1450matched'].value_counts(dropna=False))
print("\nAutistic participants in this column:")
autism_in_matched = df_clean[df_clean['autismvscontrols_1450matched'] == 1]  # or whatever value indicates autism
print(f"Count: {len(autism_in_matched)}")

In [ ]:
# Filter out the #NULL! string values to get the actual matched sample
actual_matched = df_clean[df_clean['autismvscontrols_1450matched'] != '#NULL!']
print(f"Actual matched sample size: {len(actual_matched)}")

# Now count autistic vs controls
autism_matched = actual_matched[actual_matched['autismvscontrols_1450matched'].isin([1.0, 1.00])]
controls_matched = actual_matched[actual_matched['autismvscontrols_1450matched'] == 2.00]

print(f"Autistic in matched sample: {len(autism_matched)}")
print(f"Controls in matched sample: {len(controls_matched)}")

# The numbers should be:
# 1,140 + 310 = 1,450 autistic participants
# 1,450 control participants
# Total: 2,900 participants

print("Expected autistic (1.00 + 1.0):", 1140 + 310)
print("Expected controls (2.00):", 1450)
print("Expected total matched sample:", 1450 + 1450)

# For the matched sample only
diagnosis_cols = [f'diagnosis_{i}' for i in range(9) if f'diagnosis_{i}' in actual_matched.columns]
autism_diag_cols = [f'autism_diagnosis_{i}' for i in range(3) if f'autism_diagnosis_{i}' in actual_matched.columns]

# Apply your original logic to the matched sample
diagnosis_autistic = actual_matched[diagnosis_cols].apply(lambda row: (row == 2).any() or (row == '2').any(), axis=1)
autism_diag_autistic = actual_matched[autism_diag_cols].apply(lambda row: row.isin([1, 2, 3]).any() or row.isin(['1', '2', '3']).any(), axis=1)
autistic_participants_matched = diagnosis_autistic | autism_diag_autistic

print(f"Autistic participants in matched sample (by diagnosis): {autistic_participants_matched.sum()}")

In [ ]:
# Check what values are actually in the matching column for autistic participants
autism_by_diagnosis = actual_matched[autistic_participants_matched]
print("Matching column values for participants identified as autistic by diagnosis:")
print(autism_by_diagnosis['autismvscontrols_1450matched'].value_counts())

# Check what values are in the matching column for all matched participants
print("\nAll matching column values in matched sample:")
print(actual_matched['autismvscontrols_1450matched'].value_counts())

# The issue might be that 1.00 and 1.0 are being treated differently
print("Data types of matching column values:")
print(actual_matched['autismvscontrols_1450matched'].apply(type).value_counts())

# Try a more robust way to identify autistic participants
autism_1_0 = actual_matched[actual_matched['autismvscontrols_1450matched'] == 1.0]
autism_1_00 = actual_matched[actual_matched['autismvscontrols_1450matched'] == 1.00]
autism_1_float = actual_matched[actual_matched['autismvscontrols_1450matched'].astype(float) == 1.0]

print(f"Value 1.0 (exact): {len(autism_1_0)}")
print(f"Value 1.00 (exact): {len(autism_1_00)}")
print(f"Value 1.0 (as float): {len(autism_1_float)}")

# Check if participants with matching column = 1.0/1.00 have autism diagnoses
autism_by_matching = actual_matched[actual_matched['autismvscontrols_1450matched'].isin([1.0, 1.00])]
print(f"Participants with matching column indicating autism: {len(autism_by_matching)}")

# Check their diagnosis data
diagnosis_autistic_matching = autism_by_matching[diagnosis_cols].apply(lambda row: (row == 2).any() or (row == '2').any(), axis=1)
autism_diag_autistic_matching = autism_by_matching[autism_diag_cols].apply(lambda row: row.isin([1, 2, 3]).any() or row.isin(['1', '2', '3']).any(), axis=1)
autistic_by_both = diagnosis_autistic_matching | autism_diag_autistic_matching

print(f"Of those, how many have autism diagnoses: {autistic_by_both.sum()}")